In [5]:
from datasets import load_dataset
import pandas as pd
import numpy as np

## data load

In [6]:
all_data = load_dataset('csv',data_files='./order_speech_ko1000.csv',split='train')

Using custom data configuration default-f49502427ad3afb2
Reusing dataset csv (C:\Users\AI_server\.cache\huggingface\datasets\csv\default-f49502427ad3afb2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


## data cleaning

In [7]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [8]:
remove_spectial_char_data = all_data.map(remove_special_characters)

  0%|          | 0/1000 [00:00<?, ?ex/s]

## Create Wav2Vec2CTCTokenizer

In [9]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [10]:
import torch

In [11]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [12]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [13]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [14]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [15]:
import librosa

In [18]:
sr = 16000

In [21]:

def voice_sep(sig):
    sig = np.array(sig).flatten()
    S_full, phase = librosa.magphase(librosa.stft(sig))
    S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))
    S_filter = np.minimum(S_full, S_filter)
    margin_v = 2
    power = 2
    mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)
    S_foreground = mask_v * S_full
    y_foreground = librosa.istft(S_foreground * phase)
    return y_foreground

def load_audio(batch):
    batch['array'],_ = librosa.load('./dataset/audio/'+batch['filename'],sr=16000)
    # batch['array'] = voice_sep(batch['array'])
    
    return batch

In [22]:
audio_data = remove_spectial_char_data.map(load_audio)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [26]:
type(audio_data[0]['array'])

list

## Data augmetation(RIR applied)

In [46]:
rir_raw,_ = librosa.load('./outputs/rir_data.wav',sr)

def rir_applied(batch):
    rir = torch.from_numpy( rir_raw[int(sr * 2.44) : int(sr * 2.7)].reshape(1,-1) )
    rir = rir / torch.norm(rir, p=2)
    rir = torch.flip(rir,[1])

    speech = torch.from_numpy(np.array(batch['array'],dtype=np.float32).reshape(1,-1))

    speech_ = torch.nn.functional.pad(speech, (rir.shape[1] - 1, 0))
    # print(speech.dtype)
    # print(speech_.dtype)
    # print(rir.dtype)
    augmented = torch.nn.functional.conv1d(speech_[None, ...], rir[None, ...])[0]
    batch['array'] = augmented.reshape(-1)
    return batch

In [47]:
rir_applied_audio_data = audio_data.map(rir_applied)

  0%|          | 0/1000 [00:00<?, ?ex/s]

## Preprocess Data

In [48]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [49]:
order_voice = rir_applied_audio_data.map(
    prepare_dataset,
    remove_columns=remove_spectial_char_data.column_names,
    # num_proc=4
)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [50]:
order_voice

Dataset({
    features: ['array', 'input_values', 'labels'],
    num_rows: 1000
})

# Training

## Set-up Trainer

In [51]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [52]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [53]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [54]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [55]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.weight', 'project_q.bias', 'project_q.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [56]:
model.freeze_feature_extractor()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [57]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [58]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-ko-demo",
  output_dir="./wav2vec2-large-xlsr-ko-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=80,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [59]:
from transformers import Trainer
from torch.utils.data import random_split

ds_size = len(order_voice)
train_size = int(ds_size*0.8)
val_size = ds_size - train_size
train_ds, val_ds = random_split(order_voice,[train_size,val_size])

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [60]:
torch.cuda.is_available()

True

In [61]:
hist = trainer.train()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 800
  Num Epochs = 80
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2000


  0%|          | 0/2000 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: array. If array are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.


{'loss': 52.0827, 'learning_rate': 4.2e-06, 'epoch': 0.4}
{'loss': 50.2976, 'learning_rate': 1.02e-05, 'epoch': 0.8}
{'loss': 50.924, 'learning_rate': 1.6199999999999997e-05, 'epoch': 1.2}
{'loss': 49.8284, 'learning_rate': 2.2199999999999998e-05, 'epoch': 1.6}
{'loss': 48.479, 'learning_rate': 2.7599999999999997e-05, 'epoch': 2.0}
{'loss': 45.0288, 'learning_rate': 3.36e-05, 'epoch': 2.4}
{'loss': 38.4044, 'learning_rate': 3.96e-05, 'epoch': 2.8}
{'loss': 29.8678, 'learning_rate': 4.56e-05, 'epoch': 3.2}
{'loss': 25.4932, 'learning_rate': 5.1599999999999994e-05, 'epoch': 3.6}


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


...

  0%|          | 0/25 [00:00<?, ?it/s]

Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-2000
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-2000\config.json


{'eval_loss': 2.384470224380493, 'eval_wer': 1.0407960199004975, 'eval_runtime': 11.1086, 'eval_samples_per_second': 18.004, 'eval_steps_per_second': 2.251, 'epoch': 80.0}


Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-2000\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-2000\preprocessor_config.json


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\caffe2\serialize\inline_container.cc:300] . unexpected pos 1587657280 vs 1587657168

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.